# Uncertainty propagation

In [ ]:
import numpy as np
import importlib
#import numba as nb

import plotly.offline as pyo
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as pyt

from tqdm import tqdm
import json_tricks

pyo.init_notebook_mode(connected=True) 

In [ ]:
import plotly.io as pio

In [ ]:
from variancedecomposition.model import *
from variancedecomposition.distributions import *
from variancedecomposition.plot import *

distributions = distributionsCosts

In [ ]:
def f05(CO2):
    return f(CO2, betaPERT_iCDF(0.05, 0, pStar, 1))

def f95(CO2):
    return f(CO2, betaPERT_iCDF(0.95, 0, pStar, 1))

## Model parameters

### $TCRE$: Climate sensitivity

In [ ]:
distributionsPinkPlume = [
    distributions[0], # TCRE
    lambda N: sample_normal(N, 0, 0.21),  # T1870 (initial model uncertainty)
    lambda N: sample_normal(N, 0,  0),  # sigma_nonCO2
]

In [ ]:
def createPinkPlumeValue (T, N, distributions):
    # Sample the CO2 distribution
    sample = sampleCO2value(T, N, distributions)
    
    # Calculate the 5%, 66% and 95% points
    sortedsamples = np.sort(sample)
    percent05 = sortedsamples[int(0.05 * len(sortedsamples))]
    percent66 = sortedsamples[int(0.66 * len(sortedsamples))]
    percent95 = sortedsamples[int(0.95 * len(sortedsamples))]
    median = np.median(sortedsamples)
    
    #pyo.iplot([go.Histogram(x=sample)])
    
    return (percent05, percent66, percent95, median)

In [ ]:
TvaluesPlume = np.linspace(-0.5, 5.5, 20)
plumeValues = np.array([
    createPinkPlumeValue(T, 100000, distributionsPinkPlume)
    for T in tqdm(TvaluesPlume)
])

In [ ]:
def createPinkPlume (withbuttons=True):
    
    data = [
        go.Scatter(x=plumeValues[:,0], y=TvaluesPlume, name='max', line=dict(color='rgba(0,0,0,0)', width=0)),
        go.Scatter(x=plumeValues[:,3], y=TvaluesPlume, name='mode', line=dict(color='red', width=4), fill='tonexty', fillcolor='rgba(255,0,0,.3)'),
        go.Scatter(x=plumeValues[:,2], y=TvaluesPlume, name='min', line=dict(color='rgba(0,0,0,0)', width=0), fill='tonexty', fillcolor='rgba(255,0,0,.3)'),
        #go.Scatter(x=[-0.5,10],y=[0.909+.15,0.909+.15], line=dict(color='black', width=0), name='T 2016'),
        #go.Scatter(x=[-0.5,10],y=[0.909,0.909], line=dict(color='rgb(0,0,70)', width=2), name='T 2016', fill='tonexty', fillcolor='rgba(0,0,70,.3)'),
        #go.Scatter(x=[-0.5,10],y=[0.909-.15,0.909-0.15], line=dict(color='black', width=0), name='T 2016', fill='tonexty', fillcolor='rgba(0,0,70,.3)')
    ]
    #data = []
    
    def bgimage(opacity=1):
        return [ dict(
            source = 'wg1noaxes.png',
            xref = 'x',
            yref = 'y',
            x = -0.6,
            y = 5.6,
            sizex = 10.155,
            sizey = 6.557,
            sizing = 'stretch',
            layer = 'below',
            opacity = opacity
            )
        ]
    
    updatemenus = [
        dict(type='buttons',
            active=2,
            buttons=[
                dict(label='Only IPCC plot', method='update', args = [{'visible': [False] * 3}, {'images': bgimage(1)}]),
                dict(label='Only model plume', method='update', args = [{'visible': [True] * 3}, {'images': []}]),
                dict(label='Both', method='update', args = [{'visible': [True] * 3}, {'images': bgimage(.75)}]),
            ]
        )
    ]
    
    width = 850
    
    if not withbuttons:
        updatemenus = []
        width = 750
    
    layout = go.Layout(
        xaxis=dict(title='Emissions (TtCO2)', range=[-0.3, 9.5], showgrid=True),
        yaxis=dict(title='Temperature change', range=[-0.2,5.5], showgrid=True),
        images = bgimage(.75),
        updatemenus = updatemenus,
        showlegend = False,
        width = width,
        height = 550,
        margin=dict(
            t=20,
            b=40,l=50,r=10
        )
    )
    
    fig = go.Figure(data = data, layout = layout)
    pyo.iplot(fig, image='', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False, filename='plumeAndModel')

createPinkPlume(True)

### $\sigma_{non\ CO2}$: variance due to non-CO2 emissions

In [ ]:
TempChangeFromCO2 = np.loadtxt(open("Parameter estimations/TempChangeFromCO2.csv", "rb"), delimiter=",")
TempChangeFromNonCO2 = np.loadtxt(open("Parameter estimations/TempChangeFromNonCO2.csv", "rb"), delimiter=",")

In [ ]:
trace2nonCO2 = go.Scatter(x=TempChangeFromNonCO2[:,0]/1000, showlegend=False, y=TempChangeFromNonCO2[:,1], mode='markers', name='From non-CO₂', marker=dict(color='rgba(0,0,0,.8)'))

CO2values = np.linspace(0, 8.5, 100)
trace3anonCO2 = go.Scatter(x=CO2values, y=0.1473 + 0.0875 * CO2values, showlegend=False, mode='lines', line=dict(width=3, color='rgba(63, 158, 71, 1)'))
trace3bnonCO2 = go.Scatter(x=CO2values, y=0.1473 + 0.0875 * CO2values + 0.25, showlegend=False, mode='lines', line=dict(color='#bbb'))
trace3cnonCO2 = go.Scatter(x=CO2values, y=0.1473 + 0.0875 * CO2values - 0.19, showlegend=False, mode='lines', line=dict(color='#bbb'))
trace3dnonCO2 = go.Scatter(
    x=np.concatenate([CO2values, CO2values[::-1]]),
    y=np.concatenate([0.1473 + 0.0875 * CO2values + 0.25,0.1473 + 0.0875 * CO2values[::-1] - 0.19]),
    mode='lines',
    fill='tozerox', showlegend=False,
    fillcolor='rgba(63, 158, 71, .35)',
    line=dict(color='rgba(255,255,255,0)')
)

data = [trace2nonCO2, trace3bnonCO2, trace3cnonCO2, trace3dnonCO2, trace3anonCO2]

layout = go.Layout(
    width=400, height=330,
    xaxis=dict(title='Cumulative emissions (TtCO₂)', range=[0, 8.5]),
    yaxis=dict(title='Temp. change from non-CO₂ (°C)', range=[0, 1.1]),
    margin=dict(t=10,l=50,r=20,b=50), 
    showlegend=False,
    legend=dict(orientation='h')
)

fig = go.Figure(data, layout)

pyo.iplot(fig, image='', show_link=False, filename='TempChangeFromNonCO2', image_width=fig.layout.width, image_height=fig.layout.height)#, image='svg')

In [ ]:
fig = pyt.make_subplots(1,2,print_grid=False, subplot_titles=(
    '<b>a.</b>                                                                                                ',
    '<b>b.</b>                                                                                                '
))

data = [
    go.Scatter(x=plumeValues[:,0], y=TvaluesPlume, name='max', line=dict(color='rgba(0,0,0,0)', width=0)),
    go.Scatter(x=plumeValues[:,3], y=TvaluesPlume, name='mode', line=dict(color='red', width=3), fill='tonexty', fillcolor='rgba(255,0,0,.3)'),
    go.Scatter(x=plumeValues[:,2], y=TvaluesPlume, name='min', line=dict(color='rgba(0,0,0,0)', width=0), fill='tonexty', fillcolor='rgba(255,0,0,.3)'),
    go.Scatter(x=[-0.5,10],y=[0.909+.15,0.909+.15], line=dict(color='black', width=0), name='T 2016'),
    go.Scatter(x=[-0.5,10],y=[0.909,0.909], line=dict(color='rgb(0,0,70)', width=2), name='T 2016', fill='tonexty', fillcolor='rgba(0,0,70,.3)'),
    go.Scatter(x=[-0.5,10],y=[0.909-.15,0.909-0.15], line=dict(color='black', width=0), name='T 2016', fill='tonexty', fillcolor='rgba(0,0,70,.3)')
]

for trace in data:
    fig.append_trace(trace, 1, 1)

for trace in [trace2nonCO2, trace3bnonCO2, trace3cnonCO2, trace3dnonCO2, trace3anonCO2]:
    fig.append_trace(trace, 1, 2)

fig['layout'].update(
    xaxis1=dict(title='Emissions (TtCO2)', range=[-0.3, 8.5], showgrid=True),
    yaxis1=dict(title='Temperature change (°C)', range=[-0.2,4.8], showgrid=True),
    xaxis2=dict(title='Cumulative emissions (TtCO₂)', range=[0, 8.5]),
    yaxis2=dict(title='Temp. change from non-CO₂ (°C)', range=[-0.09, 1.1]),
    images = [dict(
        source = 'wg1noaxes.png',
        xref = 'x1',
        yref = 'y1',
        x = -0.6,
        y = 5.6,
        sizex = 10.155,
        sizey = 6.557,
        sizing = 'stretch',
        layer = 'below',
        opacity = 1
        )
    ],
    showlegend = False,
    width = 800,
    height = 350,
    margin=dict(
        t=30,
        b=40,l=50,r=10
    )
)

pyo.iplot(fig, image='', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False, filename='plumeAndModel')


### $p$: costs 

In [ ]:
costsSplittedRaw = json_tricks.load(open('Parameter estimations/CostsSplitted.json'))
costsSplitted = [np.array(a) for a in costsSplittedRaw]

indices = [0.022, 0.008, 0.023]

costsOriginal = [costsSplitted[i] * np.array([1, indices[i]]) for i in range(3)]

In [ ]:
CO2levels = np.linspace(0.5,8,100)
costsLow = f05(CO2levels)
costsHigh = f95(CO2levels)
costsMode = f(CO2levels, pStar)

In [ ]:
ar5costsRaw = np.array([[0.9851, 0.2101], [0.9993, 0.4842], [0.9743, 0.8803], [1.987, 0.6107], [1.998, 1.045], [1.995, 1.250], [2.988, 1.034], [2.982, 1.384], [2.951, 2.154], [3.951, 1.519], [3.975, 2.356], [3.953, 3.719], [4.949, 2.117], [4.940, 2.719], [4.911, 4.516], [6.992, 0.4739], [6.984, 0.9841], [6.979, 1.258], [7.975, 0.8365], [7.953, 1.050], [7.961, 1.743], [8.958, 1.283], [8.949, 1.816], [8.956, 2.554], [9.957, 1.828], [9.949, 2.300], [9.932, 4.531], [10.94, 1.962], [10.93, 2.876], [10.88, 5.716], [12.99, 0.1362], [12.99, 0.1362], [12.99, 0.1362], [13.96, 0.1866], [13.96, 0.3693], [13.97, 0.4759], [15.00, 0.3357], [15.00, 0.4576], [15.00, 0.6479], [15.97, 0.4014], [15.94, 1.117], [15.96, 2.396], [16.94, 0.7487], [16.93, 1.228], [16.92, 1.602]])
ar5gdplossesminmaxRaw = np.array([
    [1, 0.115,1.448],
    [2, 0.212,2.891],
    [3, 0.103,10.865884],
    [4, 0.515,6.338664],
    [5, 1.510,15.201658]
])
ar5consumptionlossesminmaxRaw = np.array([
    [1, -0.280745, 2.176519],
    [2, 0.106914, 4.619673],
    [3, 0.270221, 9.500628],
    [4, 0.339615, 4.467258],
    [5, 0.533799, 13.383114]
])

In [ ]:
co2concentrations = np.array([
    [685, 2570, 3340],
    [615, 1870, 2440],
    [550, 1240, 2100],
    [500, 960, 1550],
    [450, 630, 1180]
])

def processAR5costs (raw):
    n = len(raw)
    
    raw[:,0] = np.round(raw[:,0])
    
    tmp = np.reshape(raw, (3, int(n/9), 3, 2))
    
    for i in range(len(co2concentrations)):
        tmp[:,i,:,0] = np.mean(co2concentrations[i][1:])
    
    return {
        'consumptionloss': tmp[0],
        'gdplosses': tmp[1],
        'abatementcosts': tmp[2]
    }

In [ ]:
AR5processed = processAR5costs(ar5costsRaw)

In [ ]:
def indexAR5costs (costs, median):
    return costs / median[2]

In [ ]:
indexedAR5gdplosses = [indexAR5costs(AR5processed['gdplosses'][:,i,1],AR5processed['gdplosses'][:,1,1]) for i in range(3)]
#print(indexedAR5gdplosses)
indexedAR5consumptionloss = [indexAR5costs(AR5processed['consumptionloss'][:,i,1],AR5processed['consumptionloss'][:,1,1]) for i in range(3)]

In [ ]:
ar5gdplossesmin = ar5gdplossesminmaxRaw[:,0:2].copy()
ar5gdplossesmin[:,0] = np.mean(co2concentrations[:,1:], axis=1)
ar5gdplossesmin[:,1] = indexAR5costs(ar5gdplossesmin[:,1], AR5processed['gdplosses'][:,1,1])

ar5gdplossesmax = ar5gdplossesminmaxRaw[:,[0,2]].copy()
ar5gdplossesmax[:,0] = np.mean(co2concentrations[:,1:], axis=1)
ar5gdplossesmax[:,1] = indexAR5costs(ar5gdplossesmax[:,1], AR5processed['gdplosses'][:,1,1])

#print([list(x) for x in list(ar5gdplossesmin)])
#print([list(x) for x in list(ar5gdplossesmax)])

In [ ]:
ar5consumptionlossmin = ar5consumptionlossesminmaxRaw[:,0:2].copy()
ar5consumptionlossmin[:,0] = np.mean(co2concentrations[:,1:], axis=1)
ar5consumptionlossmin[:,1] = indexAR5costs(ar5consumptionlossmin[:,1], AR5processed['consumptionloss'][:,1,1])

ar5consumptionlossmax = ar5consumptionlossesminmaxRaw[:,[0,2]].copy()
ar5consumptionlossmax[:,0] = np.mean(co2concentrations[:,1:], axis=1)
ar5consumptionlossmax[:,1] = indexAR5costs(ar5consumptionlossmax[:,1], AR5processed['consumptionloss'][:,1,1])

In [ ]:
def createBarChart(x, minima, bottom, median, top, maxima, width=0.1, yaxis='y', show=True):
    color = 'rgba(133, 25, 121, .8)'
    mw = 0.15 # Median Width
    data = [
        go.Bar(x=x, y=minima, width=width, marker=dict(color='rgba(0,0,0,0)'), yaxis=yaxis, showlegend=False),
        go.Bar(x=x, y=bottom-minima, width=width, marker=dict(color='rgba(133, 25, 121, .5)'), showlegend=show, yaxis=yaxis, name='AR5: full range'),
        go.Bar(x=x, y=median-bottom-mw/2, width=width, marker=dict(color=color), showlegend=show, yaxis=yaxis, name='AR5: 66% CI'),
        go.Bar(x=x, y=0*x+mw, width=width, marker=dict(color='black'), yaxis=yaxis, showlegend=show, name='AR5: median'),
        go.Bar(x=x, y=top-median-mw/2, width=width, marker=dict(color=color), yaxis=yaxis, showlegend=False),
        go.Bar(x=x, y=maxima-top, width=width, marker=dict(color='rgba(133, 25, 121, .5)'), yaxis=yaxis, showlegend=False)
    ]
    return data

In [ ]:
def combinedPlot(legend=3):

    fig = pyt.make_subplots(2,2,print_grid=False, subplot_titles=(
        '<b>a.</b>                                                                                                ',
        '<b>b.</b>                                                                                                ',
        '<b>c.</b>                                                                                                                                                                                                 '
    ), specs=[[{}, {}], [{'colspan': 2}, None]])

    data = [
        go.Scatter(x=plumeValues[:,0], y=TvaluesPlume, showlegend=False, name='max', line=dict(color='rgba(0,0,0,0)', width=0)),
        go.Scatter(x=plumeValues[:,3], y=TvaluesPlume, showlegend=legend==1, name='Model (mode, 90% CI)', line=dict(color='red', width=3), fill='tonexty', fillcolor='rgba(255,0,0,.3)'),
        go.Scatter(x=plumeValues[:,2], y=TvaluesPlume, showlegend=False, name='min', line=dict(color='rgba(0,0,0,0)', width=0), fill='tonexty', fillcolor='rgba(255,0,0,.3)'),
        go.Scatter(x=[-0.5,10],y=[0.909+.15,0.909+.15], showlegend=False, line=dict(color='black', width=0), name='T 2010'),
        go.Scatter(x=[-0.5,10],y=[0.909,0.909], showlegend=legend==1, mode='lines', line=dict(color='rgb(0,0,70)', width=2), name='T₂₀₁₀ (mode, 90% CI)', fill='tonexty', fillcolor='rgba(0,0,70,.3)'),
        go.Scatter(x=[-0.5,10],y=[0.909-.15,0.909-0.15], showlegend=False, line=dict(color='black', width=0), name='T 2010', fill='tonexty', fillcolor='rgba(0,0,70,.3)'),
        go.Scatter(x=[0,1],y=[0.9,0.9],mode='markers',xaxis='x4',marker=dict(color='rgba(0,0,0,0)'),showlegend=False)
    ]

    plumecolor = 'rgba(52, 105, 177, .3)'

    dataCosts = [
        go.Scatter(x=costsSplitted[0][:,0]/1000, y=costsSplitted[0][:,1], showlegend=legend==3, mode='markers', name='SSP: Cost estimate', marker=dict(color='#1f77b4')),
        go.Scatter(x=costsSplitted[1][:,0]/1000, y=costsSplitted[1][:,1], showlegend=legend==3, mode='markers', name='SSP: Area under MAC', marker=dict(color='#ff7f0e')),
        go.Scatter(x=costsSplitted[2][:,0]/1000, y=costsSplitted[2][:,1], showlegend=legend==3, mode='markers', name='SSP: Consumption loss', marker=dict(color='#2ca02c')),
        go.Scatter(
            x=np.concatenate([CO2levels,CO2levels[::-1]]),
            y=np.concatenate([costsHigh,costsLow[::-1]]),
            line=dict(color='rgba(0,0,0,0)', width=0),
            name='Model: 90% CI', yaxis='y5',
            fillcolor=plumecolor, fill='toself', showlegend=legend==3
        ),
        go.Scatter(x=CO2levels, y=costsMode, line=dict(width=3,color='rgb(20,20,20)'), showlegend=legend==3, name='Model: median', yaxis='y5'),
        go.Scatter(x=costsSplitted[0][:,0]/1000, y=toRealCosts(costsSplitted[0][:,1]), yaxis='y4', mode='markers', name='Cost estimate', marker=dict(color='rgba(0,0,0,0)'), showlegend=False), 
    ]

    databars = createBarChart(
        #x=(AR5processed['gdplosses'][:,0,0]+1*0.04)/1000, 
        #minima=ar5gdplossesmin[:,1],
        #bottom=indexedAR5gdplosses[0],
        #median=indexedAR5gdplosses[1],
        #top=indexedAR5gdplosses[2],
        #maxima=ar5gdplossesmax[:,1],
        x=(AR5processed['consumptionloss'][:,0,0]+1*0.04)/1000, # +40GtCO2 for 2010 emissions. Table 6.2 uses 2011-2100 budgets
        minima=ar5consumptionlossmin[:,1],
        bottom=indexedAR5consumptionloss[0],
        median=indexedAR5consumptionloss[1],
        top=indexedAR5consumptionloss[2],
        maxima=ar5consumptionlossmax[:,1],
        
        #width=(co2concentrations[:,2]-co2concentrations[:,1])/1000.0,
        width=0.2,
        yaxis='y5',
        show=legend==3
    )

    dataCosts = dataCosts + databars
    
    
    trace2nonCO2 = go.Scatter(x=TempChangeFromNonCO2[:,0]/1000, showlegend=legend==2, y=TempChangeFromNonCO2[:,1], mode='markers', name='SSP data', marker=dict(color='rgba(0,0,0,.8)'))

    CO2values = np.linspace(0, 8.5, 100)
    trace3anonCO2 = go.Scatter(x=CO2values, y=0.1473 + 0.0875 * CO2values, name='Model (mode, 90% CI)', showlegend=legend==2, mode='lines', line=dict(width=3, color='rgba(63, 158, 71, 1)'), fill='tonexty',fillcolor='rgba(63, 158, 71, .35)')
    trace3bnonCO2 = go.Scatter(x=CO2values, y=0.1473 + 0.0875 * CO2values + 0.25, showlegend=False, mode='lines', line=dict(color='#bbb'))
    trace3cnonCO2 = go.Scatter(x=CO2values, y=0.1473 + 0.0875 * CO2values - 0.19, showlegend=False, mode='lines', line=dict(color='#bbb'))
    trace3dnonCO2 = go.Scatter(
        x=np.concatenate([CO2values, CO2values[::-1]]),
        y=np.concatenate([0.1473 + 0.0875 * CO2values,0.1473 + 0.0875 * CO2values[::-1]-0.19]),
        mode='lines',
        name='90% CI',
        fill='tozerox', showlegend=False,
        fillcolor='rgba(63, 158, 71, .35)',
        line=dict(color='rgba(255,255,255,0)')
    )



    for trace in data:
        fig.append_trace(trace, 1, 1)

    for trace in [trace2nonCO2, trace3bnonCO2, trace3anonCO2, trace3cnonCO2, trace3dnonCO2, ]:
        fig.append_trace(trace, 1, 2)

    for trace in dataCosts:
        fig.append_trace(trace, 2, 1)
    
    fig['data'][6]['xaxis'] = 'x4'
    
    fig['data'][15]['yaxis'] = 'y5'
    fig['data'][16]['yaxis'] = 'y5'
    fig['data'][17]['yaxis'] = 'y4'
    fig['data'][18]['yaxis'] = 'y5'
    fig['data'][19]['yaxis'] = 'y5'
    fig['data'][20]['yaxis'] = 'y5'
    fig['data'][21]['yaxis'] = 'y5'
    fig['data'][22]['yaxis'] = 'y5'
    fig['data'][23]['yaxis'] = 'y5'
    
    if legend == 1:
        legendpos = dict(
            x=0.02,
            y=0.53,
            orientation='h'
        )
    if legend == 2:
        legendpos = dict(
            x=0.6,
            y=0.53,
            orientation='h'
        )
    if legend == 3:
        legendpos = dict(
            x=0.75,
            y=0.1
        )
    
    shift1870to2010 = 1.93 # TtCO2
    
    fig['layout'].update(
        xaxis1=dict(title='Cumulative emissions since 1870 (TtCO₂)', range=[-0.3, 8.5], showgrid=True, side='top'),
        xaxis4=dict(
            title='Cumulative emissions since 2010  (TtCO₂)',
            range=[-0.3-1.9, 8.5-1.9],
            overlaying='x1',
            side='bottom',
            anchor='y1',
            showgrid=False,
            zeroline=False
        ),
        yaxis1=dict(title='Temperature change (°C)', range=[-0.2,4.8], domain=[0.625, 0.97], showgrid=True),
        xaxis2=dict(title='Cumulative emissions since 2010 (TtCO₂)', range=[0, 8.5]),
        yaxis2=dict(title='Temp. change from non-CO₂ (°C)', range=[-0.09, 1.1], domain=[0.625, 0.97]),
        yaxis3=dict(title='Indexed costs', range=[-0.3,7.5], domain=[0,.45]), 
        yaxis4=dict(
            title='Translated costs (trillion USD)',
            overlaying='y3',
            side='right',
            anchor='x3',
            domain=[0,.45],
            showgrid=False,
            range=[toRealCosts(-0.3),toRealCosts(7.5)]
        ),
        yaxis5=dict(range=[-0.3,7.5], overlaying='y3', anchor='x3', domain=[0,.45], showgrid=False, tickvals=[]),
        xaxis3=dict(title='Cumulative emissions since 2010 (TtCO₂)', range=[0.25,6.2], domain=[0,.65]),
        images = [dict(
            source = 'wg1noaxes.png',
            xref = 'x1',
            yref = 'y1',
            x = -0.6,
            y = 5.6,
            sizex = 10.155,
            sizey = 6.557,
            sizing = 'stretch',
            layer = 'below',
            opacity = 1
            )
        ],
        barmode='stack',
        showlegend = True,
        width = 800,
        height = 750*1.05,
        margin=dict(
            t=30,
            b=40,l=50,r=10
        ),
        legend=legendpos
    )

    pyo.iplot(fig, image='svg', filename='modelComponents1', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False, config={'displayModeBar': False})
    pio.write_image(fig, 'modelComponentsNew'+str(legend)+'.svg')

combinedPlot(3)

## Cost as function of temperature

In [ ]:
def costDistribution(T, N=10000):
    samples = [dist(N) for dist in distributions]
    
    # First sample the CO2 distribution given a temperature goal
    CO2values = CO2asfunctionofTemperature(T, samples[0], samples[1], samples[2])
    
    # Then use these values to calculate the distribution in costs
    costs = toRealCosts(f(CO2values, samples[3]))
    
    # Calculate median, 5% and 95% percentile
    median  = np.median(costs)
    
    sortedCosts = np.sort(costs)
    
    percentile05 = sortedCosts[round(0.05*N)]
    percentile95 = sortedCosts[round(0.95*N)]
    
    return (median, percentile05, percentile95, T)

In [ ]:
%%time
costRange = np.array([costDistribution(T, 100000) for T in tqdm(np.linspace(0.5,6,50))])

In [ ]:
print(costDistribution(1.5, N=100000))
print(costDistribution(2.0, N=100000))

In [ ]:
selection = costRange[:,3]>1.0
selectionMedian = selection & (costRange[:,0]>2)
selectionMin = selection & (costRange[:,1]>2)
selectionMax = selection & (costRange[:,2]>2)
pyo.iplot([
    go.Scatter(x=costRange[selectionMedian,3], y=costRange[selectionMedian,0]),
    go.Scatter(x=costRange[selectionMin,3], y=costRange[selectionMin,1]),
    go.Scatter(x=costRange[selectionMax,3], y=costRange[selectionMax,2])
])

## Sensitivity analysis (Sobol method)

In [ ]:
with open('output/relativeVariancesModel0.json') as jsonfile:
    outputSensitivity = json_tricks.load(jsonfile)

In [ ]:
Tvalues = outputSensitivity['Tvalues']
processed = processResults(outputSensitivity['results'])

In [ ]:
# def cumulativeAreaChart (xvalues, yvalues, colors, names, arrowshift=None, textcolor=None):
    
#     # Calculate cumulative values for stacked chart
#     cumulative = [
#         np.sum(yvalues[:,0:i+1],1)
#         for i in range(yvalues.shape[1])
#     ]

#     # Plot
#     traces = [
#         go.Scatter(
#             x=xvalues, y=cumulative[i],
#             #name=names[i],
#             mode='lines',
#             fill='tonexty',
#             fillcolor=colors[i % len(colors)],
#             text=[str(int(y*100))+'%' for y in yvalues[:,i]],
#             hoverinfo='x+text'
#         )
#         for i in range(yvalues.shape[1])]
    
#     if arrowshift == None:
#         arrowshift = np.zeros(yvalues.shape[1])
#     if textcolor == None:
#         textcolor = ['#000'] * yvalues.shape[1]
    
#     layout = go.Layout(
#         colorway=colors,
#         barmode='stack',
#         width=900,
#         height=900,
#         xaxis=dict(title='Temperature change (°C)', range=[np.min(xvalues),np.max(xvalues)]),
#         yaxis=dict(title='Rel. contribution to variance'),
#         font = dict(size=13),
#         margin=dict(r=235, t=30,l=60),
#         annotations=[
#             dict(
#                 x=np.max(xvalues), y=cumulative[i][-1] - yvalues[-1,i]/2,
#                 xref='x', yref='y',
#                 xanchor='left',
#                 text=names[i],
#                 showarrow=True,
#                 font=dict(color=textcolor[i], size=14),
#                 arrowhead=1, arrowsize=1, arrowwidth=2,
#                 arrowcolor=colors[i % len(colors)], ax=20, ay=arrowshift[i],
#                 borderwidth=0, bgcolor=colors[i % len(colors)]
#             ) for i in range(len(names))
#         ],
#         showlegend=False
#     )
    
#     fig = go.Figure(data=traces, layout=layout)
#     return fig

In [ ]:
namesOrig = ['TCRE', 'T2010', 'Non-CO2 var.', 'Mitig. costs']
desiredOrder = [0, 1, 3, 2]

crop_id = np.where((1.4 < Tvalues) & (Tvalues <= 5.0))[0]
Tvalues_crop = Tvalues[crop_id]

firstOrderValues = processed[0][crop_id][:,desiredOrder]
names = ['1st: '+namesOrig[i] for i in desiredOrder]

labels2nd = []
for i in range(len(distributions)):
    for j in range(i+1, len(distributions)):
        labels2nd += ['2nd: ' + namesOrig[i]+' ↔ '+namesOrig[j]]

withSecondOrder = np.concatenate([firstOrderValues, processed[1][crop_id]], 1)
names += labels2nd

withThirdOrder = np.concatenate([withSecondOrder, processed[2][crop_id]], 1)
names += ['3rd: TCRE ↔ T₂₀₁₀ ↔ σ_nonCO₂']

other_fill = np.maximum(0,1 - np.sum(withThirdOrder,1))
withOther = np.concatenate([withThirdOrder, np.transpose([other_fill])], 1)
names += ['Other']

#normalise = False

#colors = ['#1f77b4','#17becf','#bcbd22','#2ca02c','#7f7f7f','#8c564b','#9467bd']
colors = [
    'rgb(101,136,175)','rgb(136,191,186)','rgb(241,155,76)','rgb(225,108,111)',
    '#2ca02c','#17becf','#bcbd22',
    'rgb(101,136,175)','rgb(136,191,186)','rgb(241,155,76)','rgb(225,108,111)',
    'rgba(150,150,150,.5)'
]


fig = cumulativeAreaChart(
    Tvalues_crop, withOther, colors, names,
    arrowshift=[0,18,0,-12,-15,0,0,15,-12,-35,-60, -65], 
    textcolor=['#FFF']*12
)
pyo.iplot(fig, image='', filename='relativeVariance', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False)

#### For paper

In [ ]:
crop_id = np.where((1.4 < Tvalues) & (Tvalues < 4.6))[0]
Tvalues_crop = Tvalues[crop_id]

namesOrig = ['TCRE', 'T2010', 'sigma_nonCO2', 'p']
desiredOrder = [0, 1, 3, 2]
firstOrderValues = processed[0][crop_id][:,desiredOrder]
names = [namesOrig[i] for i in desiredOrder]

#labels2nd = []
#for i in range(len(distributions)):
#    for j in range(i+1, len(distributions)):
#        labels2nd += ['2nd: ' + namesOrig[i]+' ↔ '+namesOrig[j]]

#withSecondOrder = np.concatenate([firstOrderValues, processed[1][crop_id]], 1)
#names += labels2nd

#withThirdOrder = np.concatenate([withSecondOrder, processed[2][crop_id]], 1)
#names += ['3rd: TCRE ↔ T2010 ↔ sigma_nonCO2']

other_fill = np.maximum(0,1 - np.sum(firstOrderValues,1))
withOther = np.concatenate([firstOrderValues, np.transpose([other_fill])], 1)
names += ['Interaction terms']

#normalise = False

#colors = ['#1f77b4','#17becf','#bcbd22','#2ca02c','#7f7f7f','#8c564b','#9467bd']
colors = [
    'rgb(101,136,175)','rgb(136,191,186)','rgb(241,155,76)','rgb(225,108,111)',
    'rgba(150,150,150,.5)',
    '#2ca02c','#17becf','#bcbd22',
    'rgb(101,136,175)','rgb(136,191,186)','rgb(241,155,76)','rgb(225,108,111)',
    'rgba(150,150,150,.5)'
]



fig = cumulativeAreaChart(
    Tvalues_crop, withOther, colors, names,
    arrowshift=[0,10,-10,-5,0,0,0,8,-17,-35,-50, -65], 
    textcolor=['#FFF']*12
)
fig['layout'].update(
    xaxis=dict(range=[np.min(Tvalues_crop),np.max(Tvalues_crop)]),
    yaxis=dict(tickformat='%'),
    margin=dict(r=170,t=10),
    height=500,
    width=800
)
pyo.iplot(fig, image='', filename='relativeVariancePaper', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False)

### Carbon budget sensitivity

In [ ]:
def modelCarbonBudget(T, paramvalues):
    TCRE = paramvalues[:,0]
    T2010 = paramvalues[:,1]
    sigma_nonCO2 = paramvalues[:,2]
    return CO2asfunctionofTemperature(T, TCRE, T2010, sigma_nonCO2)

In [ ]:
distributionsCarbonBudget = distributions[:3]

In [ ]:
Tvalues = np.linspace(0.5,4,50)

In [ ]:
def parallelTaskCB(T):
    return [testSensitivityCB(
        1000000,
        T,
        distributionsCarbonBudget,
        modelCarbonBudget,
        [(0,1),(0,2),(1,2)],
        (0,1,2)
    ) for i in range(50)]

In [ ]:
%%time
pool = multiprocessing.Pool()
result = pool.map(parallelTaskCB, Tvalues)
pool.close()  
pool.join() 

In [ ]:
processed = processResults(result)

In [ ]:
namesOrig = ['TCRE', 'T2010', 'sigma_nonCO2']
desiredOrder = [0, 1, 2]
firstOrderValues = processed[0][:,desiredOrder]
names = [namesOrig[i] for i in desiredOrder]

#labels2nd = []
#for i in range(len(distributions)):
#    for j in range(i+1, len(distributions)):
#        labels2nd += ['2nd: ' + namesOrig[i]+' ↔ '+namesOrig[j]]

#withSecondOrder = np.concatenate([firstOrderValues, processed[1][crop_id]], 1)
#names += labels2nd

#withThirdOrder = np.concatenate([withSecondOrder, processed[2][crop_id]], 1)
#names += ['3rd: TCRE ↔ T2010 ↔ sigma_nonCO2']

other_fill = np.maximum(0,1 - np.sum(firstOrderValues,1))
withOther = np.concatenate([firstOrderValues, np.transpose([other_fill])], 1)
names += ['Interaction terms']

#normalise = False

#colors = ['#1f77b4','#17becf','#bcbd22','#2ca02c','#7f7f7f','#8c564b','#9467bd']
colors = [
    'rgb(101,136,175)','rgb(136,191,186)','rgb(225,108,111)',
    'rgba(150,150,150,.5)','rgb(241,155,76)',
    '#2ca02c','#17becf','#bcbd22',
    'rgb(101,136,175)','rgb(136,191,186)','rgb(241,155,76)','rgb(225,108,111)',
    'rgba(150,150,150,.5)'
]



fig = cumulativeAreaChart(
    Tvalues, withOther, colors, names,
    arrowshift=[0,37,20,0], 
    textcolor=['#FFF']*12
)
fig['layout'].update(
    xaxis=dict(range=[np.min(Tvalues),np.max(Tvalues)]),
    yaxis=dict(tickformat='%'),
    margin=dict(r=170,t=10),
    height=500,
    width=800
)
pyo.iplot(fig, image='svg', filename='relativeVarianceCarbonBudget', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False)

In [ ]:
(0.075/0.135)**2

In [ ]:
pyo.iplot(go.Figure(
    data=[
        go.Scatter(x=Tvalues,y=firstOrderValues[:,1]/firstOrderValues[:,2]),
        go.Scatter(x=Tvalues,y=Tvalues*0 + (0.075/0.135)**2)
    ],
    layout=go.Layout(yaxis=dict(range=[0,1]))
))

### With total

In [ ]:
names = ['TCRE', 'T2010', 'sigma_nonCO2', 'p']

onlyfirst = True
normalise = True

colors = ['#1f77b4','#17becf','#bcbd22','#2ca02c','#7f7f7f','#8c564b','#9467bd']
colors = ['rgb(101,136,175)','rgb(136,191,186)','rgb(241,155,76)','rgb(225,108,111)']

firstOrTotal = 0 if onlyfirst else 1

# Normalise data to 1
normalised = [
    results_np[:,i,firstOrTotal] / (1 + normalise * (-1 + np.sum(results_np[:,:,firstOrTotal],1)))
    for i in range(len(distributions))
]

# Use the desired order
desiredOrder = [0, 1, 3, 2]
normalised_ordered = np.array([normalised[i] for i in desiredOrder])
names_ordered = [names[i] for i in desiredOrder]

# Calculate cumulative values for stacked chart
cumulative = [
    np.sum(normalised_ordered[0:i+1,:],0)
    for i in range(len(distributions))
]

# Plot
traces = [
    go.Scatter(
        x=Tvalues, y=cumulative[i],
        name=names_ordered[i],
        mode='lines',
        fill='tonexty',
        #fillcolor=colors[i],
        text=[str(int(y*100))+'%' for y in normalised_ordered[i]],
        hoverinfo='x+text'
    )
    for i in range(len(distributions))]

layout = go.Layout(
    colorway=colors,
    barmode='stack',
    width=700,
    height=500,
    xaxis=dict(title='Temperature change'),
    yaxis=dict(title='Rel. contribution to variance')
)

pyo.iplot(go.Figure(data=traces, layout=layout))

In [ ]:
pyo.iplot(go.Figure(data=[
    go.Scatter(x=Tvalues, y=normalised_ordered[i], name=names_ordered[i], line=dict(shape='spline'))
    for i in range(len(names_ordered))
], layout=layout))

## Carbon budgets

In [ ]:
def plotCarbonBudget (sampleValues, T, maxy, colors=['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd'], budgetsInp=None, plot=True, confid=0.67):
    traces = []
    budgets = []
    
    for i in range(len(T)):
        # Calculate point above which 66% of sample points occur
        # First order the sample values
        sortedsamples = np.sort(sampleValues[i])
        likely = sortedsamples[int((1-confid) * len(sortedsamples))]

        Tstring = '%.1f°C' % (T[i])

        print('Likely carbon budget for T=%s: %.4f Tt CO2' % (Tstring,likely))
        
        likelyLine = likely if budgetsInp == None else budgetsInp[i]

        # Plot the histogram
        traceHistogram = go.Histogram(x=sampleValues[i], histnorm='probability density', marker=dict(color=colors[i], opacity=0.85), xbins=dict(start=0.0, end=7.5,size=7.0/100), name='PDF for T='+Tstring)
        traceLikelyline = go.Scatter(x=[likelyLine, likelyLine], y=[-0.2, maxy[i]], name='Likely budget', marker=dict(color=colors[i]), line=dict(color='black', width=1.5), mode='lines', showlegend=i==len(T)-1)
        
        traces += [traceHistogram, traceLikelyline]
        
        budgets += [[T[i], likely]]
    
    layout = go.Layout(
        #title='CO2 distribution for T='+str(T)+'°C',
        xaxis = dict(range = [0, 7.5], title = 'Carbon budget (Tt CO₂)'),
        height = 450,
        width = 800,
        font = dict(size=13),
        yaxis = dict(range = [0,1.32], title = 'Probability density'),
        #bargap = 0,
        margin = dict(l = 60, t = 20, b = 60),
        barmode = 'overlay'
    )
    
    if plot:
        fig = go.Figure(data=traces, layout=layout)
        pyo.iplot(fig, image='', filename='carbonBudgetsDist', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False)
    
    return (np.array(budgets), traces)

In [ ]:
testvaluesT = np.array([1.5, 2, 2.5, 3, 3.5])
testvalues = [sampleCO2value(T, 100000, distributions) for T in tqdm(testvaluesT)]

In [ ]:
# Manually adjusted carbon budgets

# budget1_5degrees = 0.830*1.0075
# budget2_0degrees = 1.601*1.006
# budget2_5degrees = 2.367*1.0045
# budget3_0degrees = 3.123*1.0055
# budget3_5degrees = 3.884*1.0042

In [ ]:
# Manually adjusted carbon budgets

budget1_5degrees = 0.7761*1.0185
budget2_0degrees = 1.4750*1.0161
budget2_5degrees = 2.1659*1.015
budget3_0degrees = 2.8528*1.015
budget3_5degrees = 3.5361*1.016

In [ ]:
# First estimate:
budgetsCO2, budgetsTraces = plotCarbonBudget(
    testvalues,
    plot=False,
    T=testvaluesT,
    maxy=[1.4,1.4,1.4,1.4, 1.4],
    budgetsInp=None,
    colors=['#A50F2D','#E32C2C','#F97742','#FCB35A','#FDE493']
)

#### Adjust carbon budgets

Since carbon budgets should be calculated as inverse problems, use binary search to fine tune the carbon budgets found above

In [ ]:
def sampleTvalues (CO2, N, distributions):
    # Sample data points
    sample = dict(
        TCRE = distributions[0](N),
        T2010 = distributions[1](N),
        sigma_nonCO2 = distributions[2](N)
    )

    # CO2 values
    Tvalues = TemperatureasfunctionofCO2(CO2, sample['TCRE'], sample['T2010'], sample['sigma_nonCO2'])
    
    return Tvalues

In [ ]:
def calcLikelyTemperature (Tvalues):
    """
    Returns the temperature value such that 66% of values are smaller or equal to this value
    """
    TvaluesSorted = np.sort(Tvalues)
    return TvaluesSorted[round(0.66 * len(Tvalues))]

In [ ]:
def binarySearchRoot (fct, xmin, xmax):
    tol = 0.0005
    maxnum = 25
    
    num = 0
    while abs(fct(xmin) - fct(xmax)) > tol and num < maxnum:
        xmid = (xmax + xmin) / 2
        if fct(xmid) > 0:
            xmax = xmid
        else:
            xmin = xmid
        num += 1
        print("Iteration %i. Current function value: %.7f" % (num, fct(xmid)))
    return xmid

In [ ]:
def checkCarbonbudget(T_target, budget):
    temps = [calcLikelyTemperature(sampleTvalues(budget, 50000, distributions)) for i in range(50)]
    T = np.mean(temps)
    return T - T_target

In [ ]:
firstguess = budgetsCO2[0,1]
budget1_5degrees = binarySearchRoot(lambda x: checkCarbonbudget(1.5, x), firstguess, 1.05 * firstguess)
print(budget1_5degrees)

In [ ]:
firstguess = budgetsCO2[1,1]
budget2_0degrees = binarySearchRoot(lambda x: checkCarbonbudget(2.0, x), firstguess, 1.05 * firstguess)
print(budget2_0degrees)

In [ ]:
firstguess = budgetsCO2[2,1]
budget2_5degrees = binarySearchRoot(lambda x: checkCarbonbudget(2.5, x), firstguess, 1.05 * firstguess)
print(budget2_5degrees)

In [ ]:
firstguess = budgetsCO2[3,1]
budget3_0degrees = binarySearchRoot(lambda x: checkCarbonbudget(3.0, x), firstguess, 1.05 * firstguess)
print(budget3_0degrees)

In [ ]:
firstguess = budgetsCO2[4,1]
budget3_5degrees = binarySearchRoot(lambda x: checkCarbonbudget(3.5, x), firstguess, 1.05 * firstguess)
print(budget3_5degrees)

In [ ]:
# First estimate:
# budgetsCO2, budgetsTraces = plotCarbonBudget(
#     testvalues,
#     plot=False,
#     T=testvaluesT,
#     maxy=[1.4,1.4,1.4,1.4, 1.4],
#     budgetsInp=[budget1_5degrees, budget2_0degrees, budget2_5degrees, budget3_0degrees, budget3_5degrees],
#     colors=['#A50F2D','#E32C2C','#F97742','#FCB35A','#FDE493'],
#     confid=0.05
# )

In [ ]:
# First estimate:
# budgetsCO2, budgetsTraces = plotCarbonBudget(
#     testvalues,
#     plot=False,
#     T=testvaluesT,
#     maxy=[1.4,1.4,1.4,1.4, 1.4],
#     budgetsInp=[budget1_5degrees, budget2_0degrees, budget2_5degrees, budget3_0degrees, budget3_5degrees],
#     colors=['#A50F2D','#E32C2C','#F97742','#FCB35A','#FDE493'],
#     confid=0.95
# )

In [ ]:
for i in np.transpose(np.array([[1.5,2,2.5,3,3.5],[budget1_5degrees,budget2_0degrees,budget2_5degrees,budget3_0degrees,budget3_5degrees]])):
    print('Target: %.1f°C\tBudget: %4.0f Gt CO2' % (i[0], i[1]*1000))

In [ ]:
for i in np.transpose(np.array([[1.5,2,2.5,3,3.5],[budget1_5degrees,budget2_0degrees,budget2_5degrees,budget3_0degrees,budget3_5degrees]])):
    print('Target: %.1f°C\tBudget: %4.0f Gt CO2' % (i[0], i[1]*1000))

## Model Output

In [ ]:
# Sample red plume
def sampleEverything(N):
    
    samples = [dist(N) for dist in distributions]
    
    CO2values = np.linspace(0.5, 6, N)
    #samples += [CO2values]
    
    Temperature_samples = TemperatureasfunctionofCO2(CO2values, samples[0], samples[1], samples[2])
    
    Cost_samples = toRealCosts(f(CO2values, samples[3]))
    
    return (CO2values,Temperature_samples, Cost_samples)

def createResultsPlot (legend=2):
    N = 6500
    outp = sampleEverything(N)
    
    useGL = False
    scatterFct = go.Scattergl if useGL else go.Scatter

    trace1 = scatterFct(x=outp[0], y=outp[1], mode='markers', marker=dict(color=outp[0], colorscale='Viridis', size=3), showlegend=False)
    trace2 = scatterFct(x=outp[0], y=outp[2], mode='markers', marker=dict(color=outp[0], colorscale='Viridis', size=3), showlegend=False)
    traceCombined = scatterFct(
        x=outp[1], y=outp[2],
        mode='markers',
        marker=dict(
            color=outp[0],
            showscale=True,
            colorscale='Viridis',#Viridis
            colorbar=dict(title='Tt CO₂', thickness=15, len=0.5, y=0.24, x=1),
            size=3
        ), showlegend=False)

    trace3 = go.Scatter(x=costRange[selectionMedian,3], y=costRange[selectionMedian,0], mode='lines', line=dict(width=2.5, color='#EEE'), showlegend=legend==3, name='Median')
    trace4 = go.Scatter(x=costRange[selectionMin,3], y=costRange[selectionMin,1], mode='lines', line=dict(width=2.5, color='#111'), showlegend=legend==3, name='90% CI')
    trace5 = go.Scatter(x=costRange[selectionMax,3], y=costRange[selectionMax,2], mode='lines', line=dict(width=2.5, color='#111'), showlegend=False)

    fig = pyt.make_subplots(2, 2, print_grid=False)

    fig.append_trace(trace1, 1, 1)
    for trace in budgetsTraces:
        trace['showlegend'] = legend == 2 if 'showlegend' not in trace else (trace['showlegend'] and legend == 2)
        fig.append_trace(trace, 1, 2)
    fig.append_trace(trace2, 2, 1)
    fig.append_trace(traceCombined, 2, 2)
    fig.append_trace(trace3, 2, 2)
    fig.append_trace(trace4, 2, 2)
    fig.append_trace(trace5, 2, 2)
    
    if (legend == 2):
        legendpos = dict(x=0.88, y=1)
    if (legend == 3):
        legendpos = dict(orientation='h', x=0.65)

    fig['layout'].update(dict(
        height = 800, width=900,
        xaxis1 = dict(title='Emissions (Tt CO₂)', range=[0,6]),
        yaxis1 = dict(title='Temp. change (°C)', range=[0,6], domain=[.565,1]),
        xaxis2 = dict(title='Emissions (Tt CO₂)', range=[0, 6.5]),
        yaxis2 = dict(title='Probability density', range=[0, 1.365], domain=[.565,1]),
        xaxis3 = dict(title='Emissions (Tt CO₂)', range=[0,6]),
        yaxis3 = dict(title='Translated costs (trillion USD)', range=[-5, 295], domain=[0, .435]),
        xaxis4 = dict(title='Temp. change (°C)', range=[0, 6.2]),
        yaxis4 = dict(title='Translated costs (trillion USD)', domain=[0,.435], range=[-5, 295]),
        showlegend = True,
        barmode = 'overlay',
        legend = legendpos,
        font = dict(size=13),
        margin=dict(
            t=30,
            b=120,l=50,r=40
        )
    ))

    pyo.iplot(fig, image='', filename='modelOutputSamples', image_width=fig.layout.width, image_height=fig.layout.height, show_link=False)

    
#createResultsPlot(2)